In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.0/613.0 kB 10.8 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import os

In [3]:
def euclidean_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

In [4]:
def grayscale_conversion(image):
  gray_image = np.zeros_like(image)
  gray_image[:,:,0] = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  gray_image[:,:,1] = gray_image[:,:,0]
  gray_image[:,:,2] = gray_image[:,:,0]

  return gray_image

In [12]:
def predict(model_path,image_path):
  model = YOLO(model_path)
  image = cv2.imread(image_path)
  # gray_image = grayscale_conversion(image)
  predictions = model.predict(save=True,show_labels=True,source=image)
  return predictions[0]

In [6]:
def clustering(centers,boxes,names):
  X=centers
  eps = 1
  min_samples = 1
  dbscan = DBSCAN(eps=eps,min_samples=min_samples)
  dbscan.fit(X)

  labels = dbscan.labels_
  core_samples = dbscan.core_sample_indices_


  n_clusters = len(set(labels)) - (1 if -1 in labels else 0)


  clusters = [0]*n_clusters

  for idx, c in enumerate(centers):

    cls = int(boxes[idx].cls.tolist()[0])
    nm = names[cls]
    clusters[labels[idx]] = [c,nm]

  return clusters


In [7]:
def column_detection(clusters,boxes,names):

  X = [cluster[0] for cluster in clusters]

  eps = 22.0*2
  min_samples = 1
  dbscan = DBSCAN(eps=eps,min_samples=min_samples)
  dbscan.fit(X)

  labels = dbscan.labels_
  core_samples = dbscan.core_sample_indices_




  n_clusters = len(set(labels)) - (1 if -1 in labels else 0)





  columns = [[] for i in range(n_clusters)]


  for idx, x in enumerate(X):
    columns[labels[idx]].append([x,clusters[idx][1]])

  return columns

In [8]:
def grading(predictions):
  boxes = predictions.boxes
  names = predictions.names
  centers=[]

  for box in boxes:

    xywh = box.xywh
    cls = int(box.cls.tolist()[0])
    center = box.xywh[0][0:2]
    center = center.numpy().tolist()
    center = [round(cent) for cent in center]

    centers.append(center)


  centers = np.array(centers)

  clusters = clustering(centers,boxes,names)
  columns = column_detection(clusters,boxes,names)

  [c.sort(key=lambda x:x[0][1]) for c in columns]

  answers = []
  for column in columns:
    answers += column

  answers = [{"question":idx+1, "answer":c[1], "center":c[0]} for idx, c in enumerate(answers)]
  answers

  return answers

In [9]:
def plot_answers(image_path,answers):
  font = cv2.FONT_HERSHEY_SIMPLEX
  font_scale = .2
  text_color = (255, 255, 255)  # Black color in BGR format
  text_thickness = 1
  image = cv2.imread(image_path)


  for idx,answer in enumerate(answers):
      cv2.putText(image, str(answer["answer"]).upper(), answer["center"], font, font_scale, text_color, text_thickness)

  cv2.imwrite("answer_plotting.jpg",image)
  cv2_imshow(image)

In [10]:
import gdown
file_url = "https://drive.google.com/uc?id=1GLY8NUGG_rEQPC0ctuf0v5UjyWciG5RS"
output_file = "model.pt"
gdown.download(file_url, output_file)

Downloading...
From: https://drive.google.com/uc?id=1GLY8NUGG_rEQPC0ctuf0v5UjyWciG5RS
To: /content/model.pt
100%|██████████| 6.24M/6.24M [00:00<00:00, 167MB/s]


'model.pt'

In [13]:
from google.colab import files

uploaded = files.upload()

image_path = list(uploaded.keys())[0]
model_path = "model.pt"

predictions = predict(model_path=model_path,image_path=image_path)
answers = grading(predictions)
# plot_answers(image_path,answers)
os.remove(image_path)
answers

Saving WhatsApp-Image-2023-06-13-at-11-25-20-PM-2-_jpeg.rf.887ad229b24d1d8796c4e1f48a072928.jpg to WhatsApp-Image-2023-06-13-at-11-25-20-PM-2-_jpeg.rf.887ad229b24d1d8796c4e1f48a072928.jpg


0: 640x640 8 as, 8 bs, 10 cs, 6 ds, 19 not answereds, 216.0ms
Speed: 3.3ms preprocess, 216.0ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict2


[{'question': 1, 'answer': 'b', 'center': array([115, 121])},
 {'question': 2, 'answer': 'b', 'center': array([115, 139])},
 {'question': 3, 'answer': 'a', 'center': array([115, 157])},
 {'question': 4, 'answer': 'a', 'center': array([115, 175])},
 {'question': 5, 'answer': 'b', 'center': array([114, 194])},
 {'question': 6, 'answer': 'not answered', 'center': array([114, 212])},
 {'question': 7, 'answer': 'c', 'center': array([114, 231])},
 {'question': 8, 'answer': 'a', 'center': array([114, 251])},
 {'question': 9, 'answer': 'd', 'center': array([114, 270])},
 {'question': 10, 'answer': 'c', 'center': array([114, 289])},
 {'question': 11, 'answer': 'd', 'center': array([113, 309])},
 {'question': 12, 'answer': 'c', 'center': array([114, 329])},
 {'question': 13, 'answer': 'd', 'center': array([113, 349])},
 {'question': 14, 'answer': 'c', 'center': array([114, 370])},
 {'question': 15, 'answer': 'not answered', 'center': array([113, 390])},
 {'question': 16, 'answer': 'b', 'center':

In [59]:
!rm -r runs

In [25]:
image = cv2.imread("/content/image0120230705125542925.jpg")
# gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

gray_image = np.zeros_like(image)
gray_image[:,:,0] = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray_image[:,:,1] = gray_image[:,:,0]
gray_image[:,:,2] = gray_image[:,:,0]

print(image.shape)
print(gray_image.shape)

(1869, 727, 3)
(1869, 727, 3)
